# Import libraries

In [1]:
import os
from collections import defaultdict
import time
import h5py
import re
import numpy as np
from torch.utils.data import DataLoader

from src.utils import *
from src.datasets import ExcavatorDataset
from src.metrics import VLAD, FisherVector
from src.evaluate import compute_and_save_ssim_matrices, compute_and_save_ssim_matrices_train_val
from src.config import TRANSFORMER, ROOT

/home/ais/.virtualenvs/similarity_metrics_of_images/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ais/.virtualenvs/similarity_metrics_of_images/lib/python3.12/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.23 (you have 1.4.22). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
root = ROOT

In [3]:

train_dataset = ExcavatorDataset(return_type='image+mask+path', purpose='train')
val_dataset = ExcavatorDataset(return_type='image+mask+path', purpose='test')


/home/ais/Bachelorarbeit/similarity_metrics_of_images/src/datasets.py:308: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  key: torch.tensor(value / 255.0, dtype=torch.float32)


# Load k-means and GMM models

In [5]:
k_means_models = [
    model for model in os.listdir(rf'{root}/models/pickle_model_files') if 'k_means' in model
]
print("KMeans models:", k_means_models)
gmm_model = [
    model for model in os.listdir(rf'{root}/models/pickle_model_files') if 'gmm' in model
]
print("GMM models:", gmm_model)

KMeans models: ['k_means_model_k256_root_sift.pkl', 'k_means_model_k64_sift.pkl', 'k_means_model_k16_root_sift.pkl', 'k_means_model_k32_sift.pkl', 'k_means_model_k16_sift.pkl', 'k_means_model_k32_root_sift.pkl', 'k_means_model_k256_sift.pkl', 'k_means_model_k64_root_sift.pkl', 'k_means_model_k128_sift.pkl', 'k_means_model_k24_root_sift.pkl', 'k_means_model_k128_root_sift.pkl', 'k_means_model_k24_sift.pkl']
GMM models: ['gmm_model_k32_sift.pkl', 'gmm_model_k256_sift.pkl', 'gmm_model_k256_root_sift.pkl', 'gmm_model_k64_root_sift.pkl', 'gmm_model_k16_root_sift.pkl', 'gmm_model_k24_sift.pkl', 'gmm_model_k24_root_sift.pkl', 'gmm_model_k64_sift.pkl', 'gmm_model_k32_root_sift.pkl', 'gmm_model_k16_sift.pkl', 'gmm_model_k128_root_sift.pkl', 'gmm_model_k128_sift.pkl']


# Compute and save VLAD vector matrix in `HD5` format

In [8]:
if not os.path.exists(rf'{root}/res/vlad/train'):
    os.makedirs(rf'{root}/res/vlad/train')

if not os.path.exists(rf'{root}/res/vlad/validation'):
    os.makedirs(rf'{root}/res/vlad/validation')

for model in k_means_models:
    num_clusters = int(re.findall(r'\d+', model)[0])
    vect_length = 128 * num_clusters if not 'pca' in model else 128 * num_clusters // 2
    print(f"Number of clusters: {num_clusters}, Vector length: {vect_length}")
    train_data = {}
    val_data = {}
    feature = 'root_sift' if 'root' in model else 'sift'
    for img, *_, path in train_dataset:

        vlad = VLAD(
            image=img,
            k_means=load_model(rf'{root}/models/pickle_model_files/{model}'),
            flatten=True,
            feature=feature
        ).vector
        if len(vlad) != vect_length:
            raise ValueError(f"Expected {vect_length}, got {len(vlad)}")
        path = os.path.basename(path)
        train_data[path] = vlad

    for img, *_, path in val_dataset:

        vlad = VLAD(
            image=img,
            k_means=load_model(rf'{root}/models/pickle_model_files/{model}'),
            flatten=True,
            feature=feature
        ).vector
        if len(vlad) != vect_length:
            raise ValueError(f"Expected {vect_length}, got {len(vlad)}")
        path = os.path.basename(path)
        val_data[path] = vlad

    model_name = model.replace('.pkl', '')
    save_to_hdf5(rf'{root}/res/vlad/train/{model_name}.h5', train_data)
    save_to_hdf5(rf'{root}/res/vlad/validation/{model_name}.h5', val_data)

Number of clusters: 256, Vector length: 32768
Number of clusters: 64, Vector length: 8192
Number of clusters: 16, Vector length: 2048
Number of clusters: 32, Vector length: 4096
Number of clusters: 16, Vector length: 2048
Number of clusters: 32, Vector length: 4096
Number of clusters: 256, Vector length: 32768
Number of clusters: 64, Vector length: 8192
Number of clusters: 128, Vector length: 16384
Number of clusters: 24, Vector length: 3072
Number of clusters: 128, Vector length: 16384
Number of clusters: 24, Vector length: 3072


# Compute and save Fisher vector matrix in `HD5` format

In [11]:
if not os.path.exists(rf'{root}/res/fisher/train'):
    os.makedirs(rf'{root}/res/fisher/train')

if not os.path.exists(rf'{root}/res/fisher/validation'):
    os.makedirs(rf'{root}/res/fisher/validation')

for model in gmm_model:
    num_clusters = int(re.findall(r'\d+', model)[0])
    vect_length = (2 * 128 * num_clusters + num_clusters) if not 'pca' in model else (
                                                                                                 2 * 128 * num_clusters + num_clusters) // 2
    print(f"Number of clusters: {num_clusters}, Vector length: {vect_length}")
    train_data = {}
    val_data = {}
    feature = 'root_sift' if 'root' in model else 'sift'

    for img, *_, path in train_dataset:

        fisher = FisherVector(
            image=img,
            gmm=load_model(rf'{root}/models/pickle_model_files/{model}'),
            flatten=True,
            feature=feature
        ).vector
        if len(fisher) != vect_length:
            raise ValueError(f"Expected {vect_length}, got {len(fisher)}")
        path = os.path.basename(path)
        train_data[path] = fisher

    for img, *_, path in val_dataset:

        fisher = FisherVector(
            image=img,
            gmm=load_model(rf'{root}/models/pickle_model_files/{model}'),
            flatten=True,
            feature=feature
        ).vector
        if len(fisher) != vect_length:
            raise ValueError(f"Expected {vect_length}, got {len(fisher)}")
        path = os.path.basename(path)
        val_data[path] = fisher

    model_name = model.replace('.pkl', '')
    save_to_hdf5(rf'{root}/res/fisher/train/{model_name}.h5', train_data)
    save_to_hdf5(rf'{root}/res/fisher/validation/{model_name}.h5', val_data)

Number of clusters: 32, Vector length: 8224
Number of clusters: 256, Vector length: 65792
Number of clusters: 256, Vector length: 65792
Number of clusters: 64, Vector length: 16448
Number of clusters: 16, Vector length: 4112
Number of clusters: 24, Vector length: 6168
Number of clusters: 24, Vector length: 6168
Number of clusters: 64, Vector length: 16448
Number of clusters: 32, Vector length: 8224
Number of clusters: 16, Vector length: 4112
Number of clusters: 128, Vector length: 32896
Number of clusters: 128, Vector length: 32896


# SSIM

1. Choosing kernel size for `gaussian_blur` function

Using the empirical rule, the kernel radius should span 3 times the standard deviation. Which means:

```python
kernel_radius = int(3 * sigma)
kernel_size = 2 * kernel_radius + 1 # In order that the kernel is centered around the central pixel
```

In [54]:
train_dataset = ExcavatorDataset(return_type='image+mask+path', purpose='train', transform=TRANSFORMER)
val_dataset = ExcavatorDataset(return_type='image+mask+path', purpose='test', transform=TRANSFORMER)

/home/ais/Bachelorarbeit/similarity_metrics_of_images/src/datasets.py:308: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  key: torch.tensor(value / 255.0, dtype=torch.float32)


## A) Compute SSIM Matrix within the dataset

In the code below, the data is first saved as:

```python
{
    'image_paths': List[str],
    'ssim': np.ndarray,
    'ms_ssim': np.ndarray
}
```
because of computational constraints (could takr up to 16 hours/iteration).

In [57]:
output_dir = f'{root}/res/ssim/within_train'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

batch_size = 20
gaussian_sigmas = [i for i in range(0, 12, 2)]  # [0, 2, 4, 6, 8, 10]

for sigma in gaussian_sigmas:
    compute_and_save_ssim_matrices(dataset=train_dataset,
                                   output_dir=output_dir,
                                   batch_size=batch_size,
                                   sigma=sigma)

Computing SSIM/MS-SSIM:: 100%|██████████| 79344/79344 [33:58<00:00, 38.92it/s]


Saved SSIM and MS-SSIM matrices using: 
sigma=0, kernel_size=None, compression_quality=None
Kernel size used for sigma=2: 13


Computing SSIM/MS-SSIM:: 100%|██████████| 79344/79344 [34:05<00:00, 38.79it/s]


Saved SSIM and MS-SSIM matrices using: 
sigma=2, kernel_size=13, compression_quality=None
Kernel size used for sigma=4: 25


Computing SSIM/MS-SSIM:: 100%|██████████| 79344/79344 [34:08<00:00, 38.73it/s]


...

Saved SSIM and MS-SSIM matrices using: 
sigma=6, kernel_size=37, compression_quality=None
Kernel size used for sigma=8: 49


Computing SSIM/MS-SSIM:: 100%|██████████| 79344/79344 [33:53<00:00, 39.01it/s]


Saved SSIM and MS-SSIM matrices using: 
sigma=8, kernel_size=49, compression_quality=None
Kernel size used for sigma=10: 61


Computing SSIM/MS-SSIM:: 100%|██████████| 79344/79344 [33:54<00:00, 38.99it/s]


Saved SSIM and MS-SSIM matrices using: 
sigma=10, kernel_size=61, compression_quality=None


In [53]:
# TODO: verify that ssim data is computed correctly
# ssim_data = load_hdf5(f'{root}/res/ssim/within_train/ssim_sigma0.h5')
# for i, key in enumerate(ssim_data.keys()):
#     if i == 10:
#         break
#     print(key, ssim_data[key].shape)
#     print("SSIM has shape:", ssim_data[key]['ssim'].shape)
#     print("MS-SSIM has shape:", ssim_data[key]['ms_ssim'].shape)
#     print(ssim_data[key])


KeyboardInterrupt: 

In [5]:
ssim_data = load_hdf5(f'{root}/res/ssim/train_vs_val/ssim_train_val_sigma0.h5')
ssim_data.keys()

KeyboardInterrupt: 

## B) Compute SSIM Matrix between train and validation datasets

In [ ]:
output_dir = f'{root}/res/ssim/train_vs_val'
if not os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok=True)
# TODO: compute for sigma = 10 (still missing)

batch_size = 20
gaussian_sigmas = [i for i in range(0, 12, 2)]  # [0, 2, 4, 6, 8, 10]
for sigma in gaussian_sigmas:
    compute_and_save_ssim_matrices_train_val(train_dataset=train_dataset,
                                             val_dataset=val_dataset,
                                             output_dir=output_dir,
                                             batch_size=batch_size,
                                             sigma=sigma)



Computing SSIM/MS-SSIM (val vs train):  29%|██▉       | 54/187 [38:02<4:08:51, 112.27s/it]